In [1]:
import os
import pandas as pd
import psycopg2
from dotenv import find_dotenv, load_dotenv
from pathlib import Path
import matplotlib.pyplot as plt

/Users/jonathancusick/Documents/pyenvs/baseball/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
# Set input/output locations
project_path = Path.cwd()
raw_path = str((project_path / 'data' / 'raw').resolve())

# Connect to Retrosheet database
load_dotenv(find_dotenv())
retro_db = os.getenv('RETRO_DB')
retro_user = os.getenv('RETRO_USER')
retro_pass = os.getenv('RETRO_PASS')
conn = psycopg2.connect(database=retro_db, user=retro_user, password=retro_pass)

In [3]:
query = 'select * from raw_events;'
df_all = pd.read_sql(query, conn)
df_all.head()

,game_id,visiting_team,inning,batting_team,outs,balls,strikes,pitch_sequence,vis_score,home_score,...,position_of_batter_removed_for_pinch_hitter,fielder_with_first_putout,fielder_with_second_putout,fielder_with_third_putout,fielder_with_first_assist,fielder_with_second_assist,fielder_with_third_assist,fielder_with_fourth_assist,fielder_with_fifth_assist,event_id
0,ANA201704070,SEA,1,False,0,3,2,CBSBBB,0,0,...,0,0,0,0,0,0,0,0,0,1
1,ANA201704070,SEA,1,False,0,1,2,1CB1FT,0,0,...,0,2,0,0,0,0,0,0,0,2
2,ANA201704070,SEA,1,False,1,1,1,BFX,0,0,...,0,8,0,0,0,0,0,0,0,3
3,ANA201704070,SEA,1,False,2,0,1,C1>S,0,0,...,0,0,0,0,0,0,0,0,0,4
4,ANA201704070,SEA,1,False,2,2,2,C1>S.*BBS,0,0,...,0,2,0,0,0,0,0,0,0,5


In [7]:
df_batter = df_all[df_all.ab_flag == 'T'].copy()

batter_list = []
for batter, df_orig in df_batter.groupby('batter'):
    df = df_orig.copy()
    
    df['year'] = df.game_id.iloc[0][3:7]
    df['H'] = df.hit_value.apply(lambda x: 1 if x > 0 else 0)
    df['1B'] = df.event_type.apply(lambda x: 1 if x == 20 else 0)
    df['2B'] = df.event_type.apply(lambda x: 1 if x == 21 else 0)
    df['3B'] = df.event_type.apply(lambda x: 1 if x == 22 else 0)
    df['HR'] = df.event_type.apply(lambda x: 1 if x == 23 else 0)

    batter_list.append(df)

df_batter_list = pd.concat(batter_list)

In [8]:
df_batter_list.head()

,game_id,visiting_team,inning,batting_team,outs,balls,strikes,pitch_sequence,vis_score,home_score,...,fielder_with_third_assist,fielder_with_fourth_assist,fielder_with_fifth_assist,event_id,year,H,1B,2B,3B,HR
1581,ANA201705150,CHA,1,False,2,2,0,BBX,0,0,...,0,0,0,3,2017,0,0,0,0,0
1600,ANA201705150,CHA,4,False,1,0,1,C11>X,0,0,...,0,0,0,23,2017,1,0,0,0,1
1620,ANA201705150,CHA,6,False,0,0,1,CX,3,4,...,0,0,0,43,2017,0,0,0,0,0
1639,ANA201705150,CHA,8,False,2,1,1,.CBX,3,5,...,0,0,0,61,2017,0,0,0,0,0
1659,ANA201705160,CHA,2,False,0,2,2,CBFBFX,0,0,...,0,0,0,10,2017,0,0,0,0,0


In [4]:
len(df)

191196

In [5]:
df.batter.nunique()

957

In [6]:
df.pitcher.nunique()

755

In [17]:
df[df.ab_flag == 'F'].head(20)

,game_id,visiting_team,inning,batting_team,outs,balls,strikes,pitch_sequence,vis_score,home_score,...,position_of_batter_removed_for_pinch_hitter,fielder_with_first_putout,fielder_with_second_putout,fielder_with_third_putout,fielder_with_first_assist,fielder_with_second_assist,fielder_with_third_assist,fielder_with_fourth_assist,fielder_with_fifth_assist,event_id
0,ANA201704070,SEA,1,False,0,3,2,CBSBBB,0,0,...,0,0,0,0,0,0,0,0,0,1
3,ANA201704070,SEA,1,False,2,0,1,C1>S,0,0,...,0,0,0,0,0,0,0,0,0,4
7,ANA201704070,SEA,1,True,0,1,2,CCBFFX,0,0,...,0,8,0,0,0,0,0,0,0,8
23,ANA201704070,SEA,3,True,1,3,0,BBBV,0,1,...,0,0,0,0,0,0,0,0,0,24
31,ANA201704070,SEA,4,True,1,3,2,BBSBSB,0,2,...,0,0,0,0,0,0,0,0,0,32
52,ANA201704070,SEA,7,False,0,0,2,FFH,1,3,...,0,0,0,0,0,0,0,0,0,53
72,ANA201704070,SEA,9,False,2,0,0,>B,1,5,...,0,0,0,0,0,0,0,0,0,72
94,ANA201704080,SEA,3,True,0,2,2,FCFBBB,2,1,...,0,0,0,0,0,0,0,0,0,22
106,ANA201704080,SEA,4,True,2,1,0,B*B,2,2,...,0,4,0,0,2,0,0,0,0,34
110,ANA201704080,SEA,5,False,2,3,0,*BBBB,2,2,...,0,0,0,0,0,0,0,0,0,38


In [7]:
df[(df.batter == 'seguj002') & (df.ab_flag == 'T')][['game_id', 'pitch_sequence', 'event_text', 'event_type', 'hit_value']].head(10)

,game_id,pitch_sequence,event_text,event_type,hit_value
18,ANA201704070,FBBSS,K,3,0
42,ANA201704070,X,9/F,2,0
56,ANA201704070,X,64(1)/FO/G+,2,0
74,ANA201704080,FBX,S8/L+,20,1
92,ANA201704080,CFFBFBBX,S9/L,20,1
111,ANA201704080,BX,43/G,2,0
134,ANA201704080,CX,S5/G-,20,1
166,ANA201704090,1X,S7/L+.1-2,20,1
199,ANA201704090,SBFBFFS,K,3,0
216,ANA201704090,..BSSBS,K,3,0


In [8]:
df_batter = df[(df.batter == 'seguj002') & (df.ab_flag == 'T')].copy()

In [14]:
df_batter['H'] = df_batter.hit_value.apply(lambda x: 1 if x > 0 else 0)
df_batter['1B'] = df_batter.event_type.apply(lambda x: 1 if x == 20 else 0)
df_batter['2B'] = df_batter.event_type.apply(lambda x: 1 if x == 21 else 0)
df_batter['3B'] = df_batter.event_type.apply(lambda x: 1 if x == 22 else 0)
df_batter['HR'] = df_batter.event_type.apply(lambda x: 1 if x == 23 else 0)

In [15]:
df_batter[['game_id', 'pitch_sequence', 'event_text', 'event_type', 'hit_value', 'H', '1B', '2B', '3B', 'HR']].head(30)

,game_id,pitch_sequence,event_text,event_type,hit_value,H,1B,2B,3B,HR
18,ANA201704070,FBBSS,K,3,0,0,0,0,0,0
42,ANA201704070,X,9/F,2,0,0,0,0,0,0
56,ANA201704070,X,64(1)/FO/G+,2,0,0,0,0,0,0
74,ANA201704080,FBX,S8/L+,20,1,1,1,0,0,0
92,ANA201704080,CFFBFBBX,S9/L,20,1,1,1,0,0,0
111,ANA201704080,BX,43/G,2,0,0,0,0,0,0
134,ANA201704080,CX,S5/G-,20,1,1,1,0,0,0
166,ANA201704090,1X,S7/L+.1-2,20,1,1,1,0,0,0
199,ANA201704090,SBFBFFS,K,3,0,0,0,0,0,0
216,ANA201704090,..BSSBS,K,3,0,0,0,0,0,0


In [26]:
df_batter.H.sum() / len(df_batter)

0.29961832061068705

In [28]:
len(df_batter)

524